In [25]:
# Étape 1 : Créer un fichier Python à documenter

code = """
\"\"\"
Module : bank_marketing_doc

Date : Novembre 2025

Projet réalisé par:
    -Hanane Derbak
    -Yassine Chaouch
    -Fares Sghaier
    -Ghassen Ben Hassan

\"\"\"

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# =============================================================================
# 1. Chargement des données
# =============================================================================
def charger_donnees(path):
    \"\"\"Charge le fichier CSV du dataset Bank Marketing.

    - Lit le fichier CSV séparé par un point-virgule
    - Retourne un DataFrame brut

    Args:
        path (str): Chemin vers le fichier CSV.

    Returns:
        DataFrame: Données brutes chargées.
    \"\"\"

    return pd.read_csv(path, sep=";")


# =============================================================================
# 2. Nettoyage et préparation
# =============================================================================
def nettoyer_donnees(df):
    \"\"\"Réalise les nettoyages essentiels du dataset.

    - Convertit les colonnes en types appropriés
    - Normalise certaines catégories
    - Gère la valeur spéciale -1 dans 'pdays' (jamais contacté)
    - Vérifie la présence de doublons ou valeurs manquantes

    Args:
        df (DataFrame): Jeu de données brut.

    Returns:
        DataFrame: Données nettoyées.
    \"\"\"

    df = df.copy()
    df["age"] = df["age"].astype(int)
    df["duration"] = df["duration"].astype(int)
    df["campaign"] = df["campaign"].astype(int)
    return df


# =============================================================================
# 3. Mesures de dispersion
# =============================================================================
def mesures_dispersion(df):
    \"\"\"Calcule les mesures de dispersion des variables numériques.

    Variables étudiées :
    - age
    - balance
    - duration
    - campaign
    - pdays
    - previous

    Returns:
        DataFrame: Tableau count / mean / std / min / Q1 / Q2 / Q3 / max.
    \"\"\"

    colonnes = ["age", "balance", "duration", "campaign", "pdays", "previous"]
    return df[colonnes].describe()


# =============================================================================
# 4. Détection d'outliers
# =============================================================================
def detecter_outliers(df):
    \"\"\"Détecte les valeurs aberrantes via la méthode IQR.

    Méthode :
    - Q1 = 25%
    - Q3 = 75%
    - IQR = Q3 - Q1
    - Outliers si valeur < Q1 - 1.5*IQR ou valeur > Q3 + 1.5*IQR

    Returns:
        dict: {nom_variable : nombre d'outliers}
    \"\"\"

    colonnes = ["age", "balance", "duration", "campaign", "pdays", "previous"]
    outliers_counts = {}

    for col in colonnes:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr
        mask = (df[col] < lower) | (df[col] > upper)
        outliers_counts[col] = mask.sum()

    return outliers_counts


# =============================================================================
# 5. Matrice de corrélation
# =============================================================================
def matrice_correlation(df):
    \"\"\"Calcule la matrice de corrélation Pearson entre variables numériques.

    Variables utilisées : age, balance, duration, campaign, pdays, previous

    Returns:
        DataFrame: Matrice 6x6 des corrélations.
    \"\"\"

    colonnes = ["age", "balance", "duration", "campaign", "pdays", "previous"]
    return df[colonnes].corr()


# =============================================================================
# 6. Analyse bivariée
# =============================================================================
def analyse_bivariee(df):
    \"\"\"Génère des boxplots des variables numériques selon la cible y.

    Variables étudiées :
    - age
    - balance
    - duration
    - campaign
    - pdays
    - previous
    \"\"\"

    colonnes = ["age", "balance", "duration", "campaign", "pdays", "previous"]

    for col in colonnes:
        plt.figure(figsize=(6,4))
        sns.boxplot(data=df, x="y", y=col)
        plt.title(f"Boxplot de {col} selon la souscription (y)")
        plt.show()


# =============================================================================
# 7. Détection des incohérences métier
# =============================================================================
def detecter_incoherences(df):
    \"\"\"Détecte les incohérences logiques dans le dataset.

    Règles métier détectées :
    - previous > 0  → poutcome ne doit pas être 'unknown'
    - pdays = -1 → previous doit être 0
    - pdays > 0 → poutcome ne doit pas être 'unknown'
    - duration = 0 → y doit obligatoirement être 'no'
    - campaign > 1 → duration doit être > 0

    Returns:
        dict: {nom_incohérence: sous-ensemble du DataFrame concerné}
    \"\"\"

    incoh = {}
    incoh["previous>0_poutcome_unknown"] = df[(df["previous"] > 0) & (df["poutcome"] == "unknown")]
    incoh["pdays_-1_previous>0"] = df[(df["pdays"] == -1) & (df["previous"] > 0)]
    incoh["pdays>0_poutcome_unknown"] = df[(df["pdays"] > 0) & (df["poutcome"] == "unknown")]
    incoh["duration_0_y_yes"] = df[(df["duration"] == 0) & (df["y"] == "yes")]
    incoh["campaign>1_duration_0"] = df[(df["campaign"] > 1) & (df["duration"] == 0)]
    return incoh


# =============================================================================
# 8. Résumé global
# =============================================================================
def resume_global(df):
    \"\"\"Affiche un résumé complet de l'analyse du dataset Bank Marketing.

    Contient :
    - dimensions
    - aperçu des données
    - mesures de dispersion
    - corrélations
    - incohérences détectées
    \"\"\"

    print("=== Résumé global ===")
    print("Dimensions :", df.shape)
    print("\\nAperçu :")
    print(df.head())
    print("\\nMesures de dispersion :")
    print(mesures_dispersion(df))
    print("\\nCorrélations :")
    print(matrice_correlation(df))
    print("\\nIncohérences :")
    incoh = detecter_incoherences(df)
    for k, v in incoh.items():
        print(f"- {k} : {len(v)} lignes")
"""


with open('bank_marketing_doc.py', 'w', encoding='utf-8') as f:
    f.write(code)


In [8]:
!pip install pdoc3


In [26]:
!pdoc --html bank_marketing_doc.py -o docs


docs\bank_marketing_doc.html


In [27]:
from IPython.display import IFrame
IFrame("docs/bank_marketing_doc.html", width=1100, height=700)
